<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.856 · Anàlisi de sentiments i xarxes socials</p>
<p style="margin: 0; text-align:right;">Màster universitari en Ciència de dades (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudis d'Informàtica, Multimèdia i Telecomunicació</p>
</div>
</div>
<div style="width: 100%; clear: both;">
<div style="width:100%;">&nbsp;</div>

# Anàlisi de sentiments i xarxes socials
## PLA4: Representació de xarxes socials com a grafs

## Adquisició de dades de xarxes socials

Els processos d'adquisició de dades de xarxes socials són molt diversos. En aquesta unitat, veurem exemples d'adquisició de dades de xarxes socials seguint tres enfocaments diferents:

    •	la descàrrega directa de les dades,
    •	l'obtenció de dades mitjançant peticions a API de tercers,
    •	la recol·lecció de dades a partir de web crawling.

En relació amb la primera alternativa, la descàrrega directa de les dades, presentarem alguns dels repositoris de dades de xarxes socials més populars.

Pel que fa a la interacció amb API de tercers, veurem com usar llibreries Python per a interactuar amb API.


En relació amb el *web crawling*, veurem com utilitzar la llibreria [Scrapy](https://scrapy.org/) per a construir un petit web crawler que capturi dades del nostre interès.

## 1. Descàrrega directa de les dades

Algunes vegades tindrem l'oportunitat de treballar amb conjunts de dades que ja han estat recopilades anteriorment i que es troben directament disponibles per a la seva descàrrega. Serà el cas, per exemple, de dades recopilades per algun altre analista, dades cedides directament pel proveïdor de la xarxa social, conjunts de dades públiques utilitzades com a referència, etc.

Alguns dels llocs de referència per a descarregar dades de xarxes socials són:
- L'apartat dedicat a [xarxes socials del Network repository](http://networkrepository.com/soc.php) conté múltiples conjunts de dades d'aquestes xarxes i ofereix, a més, una eina per a visualitzar de manera interactiva les xarxes i estadístiques sobre elles.
- [Stanford Large Network Dataset Collection](http://snap.stanford.edu/data/), una recopilació de conjunts de dades en forma de xarxa creada per Juri Leskovec i Andrej Krevl. Conté dades de xarxes socials com Facebook, Twitter o Google+, a més d'altres tipus de dades que tenen estructura de xarxa, per exemple, xarxes de col·laboració entre científics o xarxes d'ordinadors.
- [Mark Newman network datasets](http://www-personal.umich.edu/~mejn/netdata/), és també llistat de conjunts de dades en forma de xarxa, en aquest cas recopilades per Mark Newman. Conté alguns conjunts de dades de xarxes socials, encara que la majoria pertanyen a altres tipus de xarxes.
- El repositori [KONECT de la universitat de Koblenz](http://konect.uni-koblenz.de/networks/) posa a disposició pública dades de xarxes, algunes de les quals pertanyen a xarxes socials (les trobarem dins de la categoria *social*). 

En aquests casos, la descàrrega de les dades es fa de manera directa (usant, per exemple, un navegador web, un client [ftp](https://en.wikipedia.org/wiki/File_Transfer_Protocol), un client de [BitTorrent](https://en.wikipedia.org/wiki/BitTorrent) o amb el mateix Python), i simplement caldrà llegir el fitxer descarregat per a poder començar a treballar amb les dades. Vegem un exemple molt senzill de la descàrrega directa d'un fitxer comprimit amb dades de la xarxa Google+.


In [1]:
import requests, zipfile, sys
zip_file_url = "http://nrvis.com/download/data/soc/soc-gplus.zip"

if sys.version_info[0] == 2:
    import StringIO
    # Descarreguem un fitxer comprimit amb les dades del Network Repository
    r = requests.get(zip_file_url, stream=True)

    # Extraiem el contingut del zip a la carpeta data
    z = zipfile.ZipFile(StringIO.StringIO(r.content))
elif sys.version_info[0] == 3:
    # python3
    import io
    r = requests.get(zip_file_url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
else:
    print("Unrecognized Python version")
    exit()
    
z.extractall("./data")

In [2]:
# Obrim el fitxer descomprimit i mostrem les 10 primeres línies
with open("./data/soc-gplus.edges", 'r') as f:
    for _ in range(10):
        read_data = f.readline()
        print(read_data)


% asym unweighted

% 39242 131 23591

1 2 

1 3 

1 4 

1 5 

1 6 

1 7 

1 8 

1 9 



## 2. APIs de tercers

Moltes xarxes socials posen a la disposició dels desenvolupadors una API (de l'anglès, *Application Programming Interface*) que permet interactuar amb la xarxa, sigui per descarregar continguts existents o per publicar nous continguts. En aquesta secció, veurem com podem descarregar contingut de xarxes socials a través de les seves API.

Una API és un conjunt de mètodes de comunicació entre diversos components de programari. Les API faciliten el treball d'integració de programes, ja que permeten oferir una interfície clara i ben especificada amb la qual interactuar amb una aplicació, amagant-ne els detalls de la implementació i exposant únicament funcions específiques d'interès.

La definició d'API és molt genèrica i podem trobar diverses API en molts contextos. En aquesta unitat, ens centrarem en l'ús de les API web per a l'adquisició de dades de serveis de tercers. Les API web es defineixen habitualment com un conjunt de peticions HTTP juntament amb l'especificació de l'estructura de les dades de les respostes, normalment en format JSON o XML.

L'ús de les API web està molt estès actualment per interactuar amb grans proveïdors de serveis a internet. Alguns exemples d'API populars són les de Google maps, YouTube, Spotify, Twitter o Facebook.

Per interactuar amb una web API farem una **petició HTTP**. Al seu torn, el servidor ens respondrà amb un missatge de resposta HTTP. Les peticions i respostes HTTP s'estructuren en tres parts:

* Una línia inicial de petició, que inclou l'acció que cal realitzar (el mètode de la petició) i la URL del recurs, a les peticions; i el codi d'estat i el missatge associat, en el cas de les respostes.
* La capçalera, que inclou metadades amb diverses finalitats, per exemple, per descriure'n el contingut, fer-ne l'autenticació o controlar les galetes (cookies).
* Una línia en blanc que separa la capçalera del cos.
* El cos, que pot estar buit o contenir-hi dades.

Les accions o mètodes més usats en interacció amb API web són:
* `GET`: permet obtenir informació del recurs especificat.
* `POST`: permet enviar dades al recurs especificat.
* `PUT`: carrega dades actualitzant les ja existents al recurs especificat.
* `DELETE`: elimina informació del recurs especificat.

Dos dels formats més habituals per incloure dades a les respostes de les API web són **JSON** i **XML**. Tots dos formats tenen diverses propietats en comú. En primer lloc, van ser dissenyats per ser llegits tant per humans com per ordinadors, la qual cosa els fa ideals en aquest context. En segon lloc, tots dos incorporen informació sobre l'estructura de les dades que codifiquen. Finalment, tots dos emmagatzemen les dades en text clar. No obstant això, tots dos presenten múltiples diferències.

El format **XML** (de l'anglès, Extensible Markup Language) és un llenguatge de marques que utilitza un conjunt d'etiquetes no predefinit. Els documents XML tenen un únic element arrel del qual poden penjar altres elements. Els elements es delimiten amb una etiqueta inicial i una etiqueta final. 

El format **JSON** (de l'anglès, Javascript Object Notation) és un subconjunt de la notació d'objectes Javascript. JSON es basa en dues estructures de dades, l'array i l'objecte, que serien equivalents a les llistes i diccionaris de Python que ja hem introduït. 

### 2.1. L'API de Twitter

**Twitter** és una xarxa social creada a principis de 2006 que proporciona una plataforma per a intercanviar petits missatges de text, coneguts com tweets. Tradicionalment els tweets estaven limitats a 140 caràcters, però en l'actualitat estan permesos fins a 280.

Twitter ofereix diverses  [APIs](https://developer.twitter.com/en/docs/api-reference-index) que permeten obtenir dades de la xarxa. En aquesta unitat, treballarem amb una d'aquestes, l'API REST.

Les dades retornades per les API de Twitter es troben codificades usant Javascript Object Notation (JSON). De totes maneres, usarem una llibreria de Python per a interactuar amb l'API, Tweepy, que ja s'encarrega de processar el JSON de resposta de Twitter i crea objectes de Python amb el resultat de cada consulta, per la qual cosa no haurem de tractar amb el format de les dades manualment.

### 2.1.1. Autenticació amb l'API de Twitter

Twitter requereix autenticació per a poder utilitzar la seva API. Per aquest motiu, el primer pas per a poder obtenir dades de Twitter mitjançant la seva API és aconseguir unes credencials adequades. En aquesta secció, descriurem com obtenir credencials per a accedir a l'API de Twitter.

Per començar, cal disposar d'un compte de Twitter. Per a poder executar els exemples del notebook, necessitareu per tant tenir un compte de Twitter. Podeu utilitzar el vostre compte personal, si ja en teniu, per a sol·licitar els permisos de desenvolupador que ens permetran interactuar amb l'API. En cas contrari (o si preferiu no usar el vostre compte personal), podeu crear un compte de Twitter nou. El procés és molt senzill:

1. Accedir a [Twitter](http://www.twitter.com).
2. Prémer sobre *Sign up for Twitter* i seguir les indicacions per a completar el registre..

Després, caldrà sol·licitar convertir el compte creat recentment (o el vostre compte personal) en un compte de desenvolupador. Per a fer-ho, cal seguir els passos següents:
1. Accedir al [panell de desenvolupadors de Twitter](https://developer.twitter.com/).
2. Clicar sobre *Apply*.
3. Clicar sobre *Apply for a developer account*.
3. Prémer *Continue*.
4. Indicar perquè voleu disposar d'un compte de desenvolupador (podeu explicar que esteu fent un curs d'anàlisi de xarxes socials i que farem unes proves amb l'API de Twitter).

Per a poder completar aquest procés satisfactòriament, necessitareu que el vostre compte disposi d'un número de telèfon associat verificat. En cas contrari, veureu que us apareixerà un missatge perquè verifiqueu el vostre telèfon. 

Finalment, una vegada ja disposem d'un compte de Twitter, caldrà registrar una nova aplicació. Per a fer-ho, cal seguir els passos següents:

1. Accedir al [panell de desenvolupadors de Twitter](https://developer.twitter.com/en/apps).
2. Prémer sobre *Create new app*.
3. Emplenar el formulari amb els detalls de l'aplicació. En concret, necessitareu proporcionar com a mínim els camps:
    * *App name*
    * *Application description*
    * *Website URL*
    * *Tell us how this app will be used*

El camp Website ha de contenir una URL vàlida (per exemple, l'enllaç al vostre perfil de Twitter).

Una vegada creada l'aplicació, podeu accedir a la pestanya Keys and access tokens. Allí es troben les credencials creades recentment per a la vostra aplicació, que utilitzarem per a autenticar-nos i poder utilitzar l'API de Twitter. Veureu que ja teniu les claus Consumer API keys disponibles. A més, caldrà prémer sobre Create en la secció Access token & access token secret per a obtenir també tots dos tokens. Els quatre valors s'utilitzaran per a autenticar la nostra aplicació:

* API / Consumer Key
* API / Consumer Secret
* Access Token
* Access Token Secret

### 2.1.2 Tweepy

Una de les llibreries més populars per a interactuar amb l'API de Twitter és  [Tweepy](http://www.tweepy.org/). La llibreria permet interactuar amb l'API de Twitter d'una manera senzilla, ja que encapsula els mètodes HTTP de l'API en mètodes de Python, que es poden cridar directament. Trobareu la documentació de la llibreria en el següent [enllaç](http://tweepy.readthedocs.io).

Per a simplificar la interacció amb la llibreria, crearem una funció  `get_twitter_api`, que crea un objecte de la  [classe api de tweepy](http://tweepy.readthedocs.io/en/v3.5.0/api.html#tweepy-api-twitter-api-wrapper) inicialitzant-lo amb les nostres credencials o bé retorna l'objecte creat anteriorment (tret que s'indiqui el contrari amb el paràmetre `reset`).

In [3]:
# Importem la librería tweepy
import tweepy

# Definim la variable API, que emmagatzemarà l'objecte tweepy.API inicialitzat amb les nostres credencials.
api = None

# Definim la funció get_twitter_api, que retornarà l'objecte tweepy.API. La funció crearà
# un objecte tweepy.API si aquest no s'ha creat prèviament (o si ho indiquem explícitament 
# amb reset=True) o bé retornarà l'objecte creat anteriorment.
def get_twitter_api(reset=False, consumer_key=None, consumer_secret=None, access_token=None, access_secret=None):
    global api
    if not api or reset:
        print("Initializing tweepy API...")
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_secret)
        api = tweepy.API(auth)
    return api

A continuació, crearem l'objecte API que utilitzarem per a interactuar amb Twitter, especificant les credencials de l'aplicació que hem creat anteriorment.

In [4]:
# Creem un objecte tweepy.API amb la funció get_twitter_api
# IMPORTANTE: És necessari incloure les credencials d'accés que hagueu obtingut al crear la vostre APP
# per executar la resta d'exemples del notebook.
api = get_twitter_api(consumer_key = 'tuzlLXJ7Cjcw9ysoeEhT60ENh', 
                      consumer_secret = 'cO9hqRaP94IP66ssVqHHqVyibrLMQ1et8XHbhczy2eBW6rFFiI',
                      access_token = '1029309220234186752-GikHFK2syTi2kiH0bVx8fMNhdP8IDJ',
                      access_secret = 'AJIuznR3PXwPh30GZ8YYqyQfpTyjkZrnu5bglzGUSaN5F'
                      )
api

Initializing tweepy API...


### 2.1.3 Els usuaris de Twitter

Un dels objectes bàsics de l'API de Twitter és el que representa els usuaris de la xarxa. Tots els usuaris de Twitter tenen un identificador únic, que és un valor numèric que Twitter utilitza per a fer referència a l'usuari dins de la xarxa. Encara que aquest identificador és molt pràctic per a seleccionar un usuari concret de forma inequívoca, és difícil de recordar per a les persones, per la qual cosa Twitter permet definir tant un nom d'usuari, que s'utilitzarà per a iniciar sessió a la xarxa, com un nom de perfil, que es mostrarà en el perfil i que podrà canviar-se amb el temps, si l'usuari ho desitja.

A més d'informació d'identificació, l'objecte usuari de Twitter disposa de molts altres atributs, per exemple, la url i la descripció que l'usuari inclou en el seu perfil, el nombre de seguidors i d'amics, el nombre de tweets que ha enviat, etc. En la documentació de l'API de Twitter, trobareu una descripció de tots els atributs que tenen els objectes d'[usuari](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/user-object) de l'API de Twitter. 

Vegem com podem usar el mètode [`get_user`](http://docs.tweepy.org/en/v3.5.0/api.html?highlight=get_user#API.get_user) de Tweepy per a obtenir dades d'un usuari de Twitter:

In [5]:
# Recuperem l'objecte api
api = get_twitter_api()

# Obtenim l'objecte user, que representa un usuari de Twitter
# Es pot accedir a un usuari per id d'usuari o nom d'usuari (screen_name)
user = api.get_user('twitter')

print("El tipus de dades de la variable user és: {}".format(type(user)))
print("El nom d'usuari és: {}".format(user.screen_name))
print("L'id d'usuari és: {}".format(user.id))

El tipus de dades de la variable user és: <class 'tweepy.models.User'>
El nom d'usuari és: Twitter
L'id d'usuari és: 783214


In [6]:
# Mostramos algunos atributos del usuario recuperado
print("El nombre de seguidors és: {}".format(user.followers_count))
print("El nombre d'amics es: {}".format(user.friends_count))
print("El nombre de tweets és: {}".format(user.statuses_count))

El nombre de seguidors és: 56606984
El nombre d'amics es: 35
El nombre de tweets és: 11543


In [7]:
# Visualitzem els atributs de la instància user.
print(user)

User(_api=<tweepy.api.API object at 0x10c56bb00>, _json={'id': 783214, 'id_str': '783214', 'name': 'Twitter', 'screen_name': 'Twitter', 'location': 'Everywhere', 'profile_location': None, 'description': 'What’s happening?!', 'url': 'https://t.co/TAXQpsHa5X', 'entities': {'url': {'urls': [{'url': 'https://t.co/TAXQpsHa5X', 'expanded_url': 'https://about.twitter.com/', 'display_url': 'about.twitter.com', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 56606984, 'friends_count': 35, 'listed_count': 90738, 'created_at': 'Tue Feb 20 14:35:54 +0000 2007', 'favourites_count': 6210, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': True, 'statuses_count': 11543, 'lang': None, 'status': {'created_at': 'Fri Aug 30 18:18:14 +0000 2019', 'id': 1167501799324082178, 'id_str': '1167501799324082178', 'text': '@WittLowry #TEAMWITT is on fire', 'truncated': False, 'entities': {'hashtags': [{'text': 'TEAMWITT', 'indices': [11, 20]}], 'symb

Les xarxes socials es caracteritzen perquè permeten que els seus usuaris creïn relacions explı́cites dins de la mateixa xarxa. Twitter implementa aquestes relacions explícites mitjançant les relacions de seguiment entre usuaris. Així, doncs, els usuaris de Twitter poden seguir altres usuaris de la xarxa.

Les relacions entre usuaris de Twitter no són bidireccionals, és a dir, Twitter diferencia entre la relació en la qual Alice segueix Bob i la relació en la qual Bob segueix Alice. Quan l'usuari Alice segueix Bob, diem que Alice és una seguidora o *follower* de Bob. Alhora, Bob és aleshores un amic o *friend* d'Alice.

Podem recuperar els seguidors (*followers*) d'un usuari i els usuaris als quals aquest segueix (amics o*friends*) usant l'API de Twitter: 

In [8]:
# Obtenim els identificadors dels followers i els friends de l'usuari de Twitter
followers_ids = api.followers_ids("twitter")
friends_ids = api.friends_ids("twitter")
print("L'id del primer follower és: {}".format(followers_ids[0]))
print("L'id del primer friend és: {}".format(friends_ids[0]))

# També podem recuperar els followers o friends com a objectes User de Tweepy.
followers = api.followers("twitter")
friends = api.friends("twitter")
print("El tipus de dades d'un follower és: {}".format(type(followers[0])))
print("El tipus de dades d'un friend és: {}".format(type(friends[0])))

# Així, podem accedir directament als atributs dels followers i friends de l'usuari de Twitter
a_follower = followers[0]
print("Dades del primer follower de l'usuari twitter:")
print("\tEl nom d'usuari és: {}".format(a_follower.screen_name))
print("\tL'id d'usuari és: {}".format(a_follower.id))
print("\tEl número de seguidores es: {}".format(a_follower.followers_count))
print("\tEl nombre d'amics és: {}".format(a_follower.friends_count))
print("\tEl nombre de tweets és: {}".format(a_follower.statuses_count))


L'id del primer follower és: 1168535169298550785
L'id del primer friend és: 2408170710
El tipus de dades d'un follower és: <class 'tweepy.models.User'>
El tipus de dades d'un friend és: <class 'tweepy.models.User'>
Dades del primer follower de l'usuari twitter:
	El nom d'usuari és: BenhadjMimi
	L'id d'usuari és: 1168535169298550785
	El número de seguidores es: 0
	El nombre d'amics és: 34
	El nombre de tweets és: 0


Donada una relació entre dos usuaris, també podem obtenir dades addicionals d'aquesta usant el mètode  [`showfriendship`](http://docs.tweepy.org/en/v3.5.0/api.html?highlight=show_friendship#API.show_friendship):


In [9]:
api.show_friendship(source_id=1006576427423227905, target_id=783214)

(Friendship(_api=<tweepy.api.API object at 0x10c56bb00>, _json={'id': 1006576427423227905, 'id_str': '1006576427423227905', 'screen_name': 'murcianomario', 'following': False, 'followed_by': False, 'live_following': False, 'following_received': None, 'following_requested': None, 'notifications_enabled': None, 'can_dm': False, 'blocking': None, 'blocked_by': None, 'muting': None, 'want_retweets': None, 'all_replies': None, 'marked_spam': None}, id=1006576427423227905, id_str='1006576427423227905', screen_name='murcianomario', following=False, followed_by=False, live_following=False, following_received=None, following_requested=None, notifications_enabled=None, can_dm=False, blocking=None, blocked_by=None, muting=None, want_retweets=None, all_replies=None, marked_spam=None),
 Friendship(_api=<tweepy.api.API object at 0x10c56bb00>, _json={'id': 783214, 'id_str': '783214', 'screen_name': 'Twitter', 'following': False, 'followed_by': False, 'following_received': None, 'following_requested':

### 2.1.4 Els tweets

Els **tweets** són les unitats bàsiques de comunicació entre els usuaris de Twitter. Tots els tweets tenen associat un identificador únic dins de la xarxa, i contenen el text d'aquest.

A més de l'identificador i del text que comuniquen, també contenen informació sobre el moment en què es van crear (el dia i l'hora), com també dades de l'aplicació utilitzada per l'usuari per a crear el tweet (els tweets creats amb la interfície web de Twitter contenen simplement la cadena «web» com a identificador d'aplicació d'origen), entre altres atributs. En la documentació de l'API de Twitter, trobareu una descripció de tots els atributs que tenen els objectes [tweet](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object) de l'API de Twitter. 



Vegem com podem usar el mètode  [`get_status`](http://docs.tweepy.org/en/v3.5.0/api.html?highlight=get_status#API.get_status) de Tweepy per a obtenir dades d'un usuari de Twitter:

In [10]:
# Obtenim l'objecte status, que representa un tweet.
# Es pot accedir a un tweet a través del seu id.
status = api.get_status(990518384407318528)

print("El tipus de dades de la variable status és: {}".format(type(status)))
print("L'id del tweet és: {}".format(status.id))
print("El text del tweet és: {}".format(status.text))


El tipus de dades de la variable status és: <class 'tweepy.models.Status'>
L'id del tweet és: 990518384407318528
El text del tweet és: Las Claves del RGPD en nueve Puntos! Con la entrada en vigor del RGPD las empresas deben tomarse muy en serio la pr… https://t.co/Pe4QzyUJA3


Observeu que el text del tweet està codificat amb [unicode](https://docs.python.org/2/howto/unicode.html):

In [11]:
if sys.version_info[0] == 2:
    isinstance(status.text, unicode)
elif sys.version_info[0] == 3:
    # in Python 3, strings are unicode:
    isinstance(status.text, str)

Com hem vist, els usuaris de Twitter creen relacions explícites entre ells mitjançant les relacions de seguiment. D'una manera similar, els *tweets* també poden crear relacions explícites entre ells. És el cas dels *retweets*, que són la republicació d'un *tweet*, els *quotes*, que permeten també republicar un *tweet* però afegint-hi un comentari, i els replies, que permeten respondre a un *tweet*.

In [12]:
# Recuperem un tweet que republica un altre tweet, és a dir, un retweet
a_retweet = api.get_status(1030046532064747522)

# Mostrem dades del tweet que s'està republicant
retweeted_status = a_retweet.retweeted_status
print("El tweet està republicant el tweet amb id : {}".format(retweeted_status.id))
print("El tweet està republicant un tweet de l'usuari amb id : {}".format(retweeted_status.user.id))
print("El tweet original té com a text: {}".format(retweeted_status.text))

El tweet està republicant el tweet amb id : 1029823889365450752
El tweet està republicant un tweet de l'usuari amb id : 1469101279
El tweet original té com a text: Bitcoin Q&amp;A: Why I'm against ETFs https://t.co/8mQtKJgy0L


In [13]:
# Recuperem un tweet que respon a un altre tweet, és a dir, un reply
a_reply = api.get_status(1029918429992689665)

# Mostrem a quin tweet es respon i qui ha estat el seu autor
print("El tweet està responent al tweet amb id : {}".format(a_reply.in_reply_to_status_id))
print("El tweet està responent a l'usuari amb id : {}".format(a_reply.in_reply_to_user_id))

El tweet està responent al tweet amb id : 1029894625249509376
El tweet està responent a l'usuari amb id : 2568108282


In [14]:
# Recuperem un tweet que comenta un altre tweet, és a dir, un quote
a_quote = api.get_status(1029401266999447552)

# Mostrem el tweet que es comenta
print("El tweet està comentant el tweet amb id : {}".format(a_quote.quoted_status_id))
quoted_status = a_quote.quoted_status
print("El tweet està comentant l'usuari amb id : {}".format(quoted_status.user.id))
print("El tweet original té com a text: {}".format(quoted_status.text))

El tweet està comentant el tweet amb id : 1029112398555176961
El tweet està comentant l'usuari amb id : 297564170
El tweet original té com a text: WOW! @CapitalOne Just froze my bank account (and all $309,000 of cash I had collected for an upcoming house purchas… https://t.co/khmv9vfZvp


Observeu que els tipus de dades que s'obtenen en cada cas no són sempre els mateixos, per la qual cosa caldrà anar amb compte a l'hora de treballar amb aquests mètodes. Per exemple,`retweeted_status` és un objecte tweet de Tweepy, mentre que `quoted_status` és un diccionari.

In [15]:
print("El tipus de dades de retweeted_status és: {}".format(type(retweeted_status)))
print("El tipus de dades de quoted_status és: {}".format(type(quoted_status)))

El tipus de dades de retweeted_status és: <class 'tweepy.models.Status'>
El tipus de dades de quoted_status és: <class 'tweepy.models.Status'>


A més, Tweepy no és tampoc consistent a l'hora de tractar l'accés a camps relatius a característiques no presents en el tweet, un detall que també cal tenir en compte a l'hora d'incorporar aquestes funcionalitats als nostres programes:

In [16]:
# Accedim a in_reply_to_status_id d'un retweet:
print(a_retweet.in_reply_to_status_id)

# Accedim a retweeted_status d'un reply:
try:
    a_reply.retweeted_status
except AttributeError:
    print("AttributeError")
    
# Disposem d'un booleà que ens indica si un tweet és un quote
# però aquest booleà no existeix en cas de replies i retweets
print("El tweet a_quote és un comentari: {}".format(a_quote.is_quote_status))
print("El tweet a_reply és un comentari: {}".format(a_reply.is_quote_status))

None
AttributeError
El tweet a_quote és un comentari: True
El tweet a_reply és un comentari: False


A més, els tweets obtinguts de l'API de Twitter contenen també  [entitats](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/entities-object), un conjunt de metadades addicionals que descriuen dades que s'inclouen habitualment en tweets com a etiquetes (*hashtags*), enllaços, esments a altres usuaris, fitxers incrustats, etc. Podríem obtenir aquestes dades analitzant manualment el text del tweet, però l'API ens els retorna també ja processats.


In [17]:
# Mostrem les entitats del tweet status
status.entities

{'hashtags': [],
 'symbols': [],
 'user_mentions': [],
 'urls': [{'url': 'https://t.co/Pe4QzyUJA3',
   'expanded_url': 'https://twitter.com/i/web/status/990518384407318528',
   'display_url': 'twitter.com/i/web/status/9…',
   'indices': [117, 140]}]}

Algunes vegades, els tweets contenen també informació sobre el [lloc](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/geo-objects) des del qual es van enviar a la xarxa, sigui en forma de punts exactes, indicats a partir de les seves coordenades geogràfiques, o bé a través de «llocs» delimitats sobre una zona més àmplia.  

In [18]:
# Exemple d'un tweet vinculat a un lloc de Twitter
status_with_place = api.get_status(1030787428125032448)
print(status_with_place.place)

Place(_api=<tweepy.api.API object at 0x10c56bb00>, id='006523c50dfe9086', url='https://api.twitter.com/1.1/geo/id/006523c50dfe9086.json', place_type='city', name='Quezon City', full_name='Quezon City, National Capital Region', country_code='PH', country='Republic of the Philippines', contained_within=[], bounding_box=BoundingBox(_api=<tweepy.api.API object at 0x10c56bb00>, type='Polygon', coordinates=[[[120.989705, 14.5893763], [121.1357656, 14.5893763], [121.1357656, 14.7766484], [120.989705, 14.7766484]]]), attributes={})


In [19]:
# Exemple d'un tweet amb coordenades geogràfiques
status_with_coord = api.get_status(1030808412286636034)
print(status_with_coord.coordinates)

{'type': 'Point', 'coordinates': [-74.0064, 40.7142]}


## 3. Web crawling

De vegades, necessitarem accedir a dades que estan publicades a internet però per a les quals no hi ha un fitxer que puguem descarregar ni una API a la qual puguem fer consultes. En aquests casos, pot ser útil crear un petit crawler, és a dir, un programa que rastregi les pàgines web que continguin les dades d'interès, els analitzi i els recopili en algun fitxer d'emmagatzematge persistent, perquè puguin ser usats posteriorment per a les nostres anàlisis.

<span style="color:blue">[Scrapy](httpscrapy.org/) és una llibreria de Python que proporciona un _framework_ per a l'extracció de dades de pàgines web. Podeu veure una introducció a l'ús de Scrapy i un exemple d'ús en el notebook de la Unitat 5 de l'assignatura Programació en Python.</span>

Vegem un exemple de com usar Scrapy per a extreure dades de xarxes socials extraient els contactes d'un usuari de Flickr, una xarxa centrada en la compartició d'imatges i vídeos.

Podem obtenir un llistat dels contactes d'un usuari de Flickr accedint al seu perfil. Així, per exemple, trobarem els contactes de l'usuari `egofreed` en la url següent:

https://www.flickr.com/people/egofreed/contacts

Si volem extreure els noms dels usuaris que conformen els contactes d'aquest usuari, haurem d'observar l'html d'aquesta pàgina, per veure on podem trobar aquesta informació. 

Com es pot apreciar, les dades que volem recopilar (els noms d'usuari dels contactes) es troben en múltiples atributs. Si ens fixem en el primer contacte,

podem veure com el nom d'usuari, `cristinamerz`, apareix en tres enllaços diferents. Per tant, podem seleccionar-ne qualsevol per a extreure la dada d'interès. En el nostre cas, seleccionarem l'últim dels enllaços. Així, doncs, crearem una expressió xpath que seleccioni l'atribut href d'un hipervincle (un element assenyalat amb l'etiqueta `<a>`) que té com a atribut `rel` el valor «contact» i, de les tres coincidències que obtindrem, ens quedarem únicament amb l'última. L'expressió xpath que ens permet fer aquesta selecció és:

```
//a[@rel="contact"][2]/@href
```

Una vegada tenim l'expressió xpath que selecciona les dades d'interès, ja podem programar la nostra aranya perquè els extregui.

**Nota**: Si l'execució del *crawler* us retorna un error  `ReactorNotRestartable`, reinicieu el kernel del notebook (en el menú, `Kernel` - `Restart`).


In [20]:
# Importem scrapy
import scrapy
from scrapy.crawler import CrawlerProcess

# Creem la aranya.
class uoc_flickr_spider(scrapy.Spider):
    
    # Assignem un nom a l'aranya
    name = "uoc_flickr_spider"
    
    # Indiquem la url que volem analitzar en primer lloc.
    start_urls = [
        "https://www.flickr.com/people/egofreed/contacts"
    ]

    # Definim l'analitzador
    def parse(self, response):
        # Extraiem el nom d'usuari del contacte
        for contact in response.xpath('//a[@rel="contact"][2]/@href'):
            yield {
                # Ens quedem únicament amb el nom d'usuari
                'contact_username': contact.extract()[8:-1]
            }
            
if __name__ == "__main__":

    # Creem un crawler.
    process = CrawlerProcess({
        'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
        'DOWNLOAD_HANDLERS': {'s3': None},
        'LOG_ENABLED': True
    })

    # Inicialitzem el crawler amb la nostra aranya
    process.crawl(uoc_flickr_spider)
    
    # Llancem l'aranya
    process.start()

2019-09-02 17:47:04 [scrapy.utils.log] INFO: Scrapy 1.7.2 started (bot: scrapybot)
2019-09-02 17:47:04 [scrapy.utils.log] INFO: Versions: lxml 4.2.5.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 19.2.1, Python 3.6.7 (v3.6.7:6ec5cf24b7, Oct 20 2018, 03:02:14) - [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1c  28 May 2019), cryptography 2.7, Platform Darwin-18.7.0-x86_64-i386-64bit
2019-09-02 17:47:04 [scrapy.crawler] INFO: Overridden settings: {'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2019-09-02 17:47:04 [scrapy.extensions.telnet] INFO: Telnet Password: 1279461d64d01323
2019-09-02 17:47:04 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2019-09-02 17:47:04 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloader

En el log de l'execució de Scrapy podrem veure el resultat del nostre crawling, amb els noms d'usuari dels contactes de Flickr de l'usuari `egofreed`.